In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def get_nba_db():
    """
    Creates a connection to the NBA PostgreSQL database.
    """
    host = os.getenv("NBA_DB_HOST")
    port = os.getenv("NBA_DB_PORT")
    database = os.getenv("NBA_DB_NAME")
    user = os.getenv("NBA_DB_USER")
    password = os.getenv("NBA_DB_PASSWORD")
    
    connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"
    return create_engine(connection_string)

def query(sql):
    """
    Executes SQL query against the NBA database and returns results as a pandas DataFrame.
    """
    engine = get_nba_db()
    return pd.read_sql(sql, engine)

# Function to display available tables
def list_tables():
    """Lists all tables available in the NBA database."""
    tables = query("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'")
    return tables['table_name'].tolist()

In [2]:
list_tables()

['common_player_info',
 'draft_combine_stats',
 'draft_history',
 'game',
 'game_info',
 'game_summary',
 'games',
 'inactive_players',
 'leagueschedule24_25',
 'leagueschedule25_26',
 'line_score',
 'officials',
 'other_stats',
 'play_by_play',
 'player',
 'players',
 'playerstatistics',
 'team',
 'team_details',
 'team_history',
 'team_info_common',
 'teamhistories',
 'teamstatistics']

In [38]:
query("""
WITH BaseData AS (

SELECT
firstname
,lastname
,CAST(personid AS INT)
,CAST(gameid AS INT)
,CAST(gamedate AS DATE)
,CAST(EXTRACT('week' FROM CAST(gamedate AS DATE)) AS INT) AS week
,CAST(EXTRACT('month' FROM CAST(gamedate AS DATE)) AS INT) AS month
,CAST(EXTRACT('year' FROM CAST(gamedate AS DATE)) AS INT) AS year
,CONCAT(CAST(EXTRACT('week' FROM CAST(gamedate AS DATE)) AS INT) , '-' ,CAST(EXTRACT('year' FROM CAST(gamedate AS DATE)) AS INT)) AS weekyear
,playerteamcity
,playerteamname
,opponentteamcity
,opponentteamname
,gametype
,gamelabel
,gamesublabel
,seriesgamenumber
,CAST(win AS INT)
,CAST(home AS INT)
,numminutes
,CAST(points AS INT)
,CAST(assists AS INT)
,CAST(blocks AS INT)
,CAST(steals AS INT)
,CAST(fieldgoalsmade AS INT)
,CAST(fieldgoalsattempted AS INT)
,fieldgoalspercentage
,CAST(threepointersmade AS INT)
,CAST(threepointersattempted AS INT)
,threepointerspercentage
,CAST(freethrowsmade AS INT)
,CAST(freethrowsattempted AS INT)
,freethrowspercentage
,CAST(reboundsoffensive AS INT)
,CAST(reboundsdefensive AS INT)
,CAST(reboundstotal AS INT)
,CAST(foulspersonal AS INT)
,CAST(turnovers AS INT)
,CAST(plusminuspoints AS INT)

FROM playerstatistics

)

SELECT

firstname
,lastname
,week
,playerteamname
,SUM(numminutes) AS numminutes
,SUM(points) AS points
,SUM(assists) AS assists
,SUM(blocks) AS blocks
,SUM(steals) AS steals
,SUM(fieldgoalsmade) AS fieldgoalsmade
,SUM(fieldgoalsattempted) AS fieldgoalsattempted
,SUM(threepointersmade) AS threepointersmade
,SUM(threepointersattempted) AS threepointersattempted
,SUM(freethrowsmade) AS freethrowsmade
,SUM(freethrowsattempted) AS freethrowsattempted
,SUM(reboundsoffensive) AS reboundsoffensive
,SUM(reboundsdefensive) AS reboundsdefensive
,SUM(reboundstotal) AS reboundstotal
,SUM(foulspersonal) AS foulspersonal
,SUM(turnovers) AS turnovers
,SUM(plusminuspoints) AS plusminuspoints

FROM BaseData
WHERE YEAR = '2025'
GROUP BY firstname, lastname, week, playerteamname
LIMIT 5

""")

,firstname,lastname,week,playerteamname,numminutes,points,assists,blocks,steals,fieldgoalsmade,...,threepointersmade,threepointersattempted,freethrowsmade,freethrowsattempted,reboundsoffensive,reboundsdefensive,reboundstotal,foulspersonal,turnovers,plusminuspoints
0,Aaron,Gordon,2,Nuggets,18.22,13,2,1,0,6,...,1,2,0,0,1,5,6,0,1,23
1,Aaron,Gordon,3,Nuggets,60.86,33,3,1,1,10,...,1,7,12,13,2,10,12,4,6,17
2,Aaron,Gordon,4,Nuggets,63.11,35,6,0,2,12,...,6,11,5,6,3,2,5,3,4,6
3,Aaron,Gordon,5,Nuggets,113.82,38,10,1,2,16,...,2,8,4,4,6,8,14,6,5,24
4,Aaron,Gordon,6,Nuggets,77.06,27,23,2,1,7,...,3,8,10,13,4,13,17,4,1,40
